In [1]:
import pandas as pd
from sklearn import ensemble, preprocessing, feature_extraction, metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

import the processed dataset 

In [2]:
processed_speeches = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Processed_speeches.csv')
processed_speeches['speech'] = processed_speeches['speech'].astype('str') 
processed_speeches

,member_name,sitting_date,political_party,speaker_info,speech,speech_length,binned
0,κρητικος νικολαου παναγιωτης,1989,πανελληνιο σοσιαλιστικο κινημα,προεδρευων,παρακαλείται γραμματέας βουλγαράκης αρχιεπίσκο...,43,"(25, 100]"
1,κρητικος νικολαου παναγιωτης,1989,πανελληνιο σοσιαλιστικο κινημα,προεδρευων,παρακαλείται κύριος γραμματέας ιερά σύνοδο αιθ...,81,"(25, 100]"
2,κρητικος νικολαου παναγιωτης,1989,πανελληνιο σοσιαλιστικο κινημα,προεδρευων,παρακαλώ βουλή εξουσιοδοτήσω προεδρείο επικύρω...,17,"(10, 25]"
3,κρητικος νικολαου παναγιωτης,1989,πανελληνιο σοσιαλιστικο κινημα,προεδρευων,βουλή παρέσχε ζητηθείσος εξουσιοδότηση συναίνε...,37,"(25, 100]"
4,κρητικος νικολαου παναγιωτης,1989,πανελληνιο σοσιαλιστικο κινημα,προεδρευων,εκ κυρίος συναδέλφα ψήφομαι κηρύσσομαι περαιωμ...,50,"(25, 100]"
...,...,...,...,...,...,...,...
500591,κακλαμανης μιχαηλ νικητας,2019,νεα δημοκρατια,προεδρευων,συζητώ ερώτηση υπάρξω ανοχή συντυχάκη,17,"(10, 25]"
500592,κακλαμανης μιχαηλ νικητας,2019,νεα δημοκρατια,προεδρευων,φύγω καταθέσω πρακτικά μπορέσω πάρω,16,"(10, 25]"
500593,συντυχακης δανιηλ εμμανουηλ,2019,κομμουνιστικο κομμα ελλαδας,NaN,καταστροφή προφανώς αποτέλεσμα καιρικός φαινομ...,681,"(400, 1000]"
500594,κακλαμανης μιχαηλ νικητας,2019,νεα δημοκρατια,προεδρευων,αρκώ κεντρικό κράτος δίνω χρήματα προβλέπω σύν...,101,"(100, 400]"


In [ ]:
processed_speeches.dtypes

member_name        object
sitting_date        int64
political_party    object
speaker_info       object
speech             object
speech_length       int64
binned             object
dtype: object

the dataset is very imbalanced. 

In [3]:

df = processed_speeches['political_party'].value_counts().rename_axis('unique_values').reset_index(name='counts')
df

,unique_values,counts
0,πανελληνιο σοσιαλιστικο κινημα,179384
1,νεα δημοκρατια,175572
2,συνασπισμος ριζοσπαστικης αριστερας,52341
3,κομμουνιστικο κομμα ελλαδας,25370
4,συνασπισμος της αριστερας των κινηματων και τη...,15975
5,ανεξαρτητοι (εκτος κομματος),11190
6,λαικος ορθοδοξος συναγερμος,11077
7,δημοκρατικη συμπαραταξη (πανελληνιο σοσιαλιστι...,7594
8,ανεξαρτητοι ελληνες - πανος καμμενος,5271
9,λαικος συνδεσος - χρυση αυγη,3350


In [4]:
df.set_index('unique_values',inplace=True)

In [ ]:
df

,counts
unique_values,
πανελληνιο σοσιαλιστικο κινημα,179384
νεα δημοκρατια,175572
συνασπισμος ριζοσπαστικης αριστερας,52341
κομμουνιστικο κομμα ελλαδας,25370
συνασπισμος της αριστερας των κινηματων και της οικολογιας,15975
ανεξαρτητοι (εκτος κομματος),11190
λαικος ορθοδοξος συναγερμος,11077
δημοκρατικη συμπαραταξη (πανελληνιο σοσιαλιστικο κινημα - δημοκρατικη αριστερα),7594
ανεξαρτητοι ελληνες - πανος καμμενος,5271


drop once again the parties that have less than 1000 speeches

In [5]:
processed_speeches = processed_speeches[~((processed_speeches['political_party']=='ανεξαρτητοι δημοκρατικοι βουλευτες') | (processed_speeches['political_party']=='ανεξαρτητοι (εκτος κομματος)'))]

create n samples for each of the underrepresented classes(n=30000 - Speeches_i). My goal is to have 30000 speeches per party.

In [6]:
from sklearn.utils import resample

df_majority = pd.DataFrame()
parties = processed_speeches.political_party.unique().tolist()


for party in parties:
  if df.loc[party,'counts'] < 30000:
    df_minority = processed_speeches[processed_speeches.political_party == party]
    
    # Upsample minority class
    df_minority_upsampled = resample(df_minority, 
                                    replace=True,     # sample with replacement
                                    n_samples=30000-df.loc[party,'counts'],    # to match majority class
                                    random_state=123) # reproducible results
    df_majority = pd.concat([df_majority, df_minority_upsampled])
    
# Display new class counts
df_majority.political_party.value_counts()

πολιτικη ανοιξη                                                                    28728
το ποταμι                                                                          28226
ανεξαρτητοι ελληνες εθνικη πατριωτικη δημοκρατικη συμμαχια                         28130
δημοκρατικη αριστερα                                                               28130
ενωση κεντρωων                                                                     27182
δημοκρατικο κοινωνικο κινημα                                                       26841
λαικος συνδεσος - χρυση αυγη                                                       26650
ανεξαρτητοι ελληνες - πανος καμμενος                                               24729
δημοκρατικη συμπαραταξη (πανελληνιο σοσιαλιστικο κινημα - δημοκρατικη αριστερα)    22406
λαικος ορθοδοξος συναγερμος                                                        18923
συνασπισμος της αριστερας των κινηματων και της οικολογιας                         14025
κομμουνιστικο κομμα ε

In [7]:
upsampled = processed_speeches.copy()
upsampled = pd.concat([df_majority, upsampled ])
upsampled

,member_name,sitting_date,political_party,speaker_info,speech,speech_length,binned
26354,δρεττακης γεωργιου εμμανουηλ,1991,συνασπισμος της αριστερας των κινηματων και τη...,προεδρευων,κύριος υπουργός ανέγνωσε προηγουμένως μπω τροπ...,126,"(100, 400]"
216538,κουβελης ευαγγελου φωτιος-φανουριος,2003,συνασπισμος της αριστερας των κινηματων και τη...,NaN,υφυπουργέ οφείλω και απόλυτα ειλικρινής ογα ερ...,303,"(100, 400]"
103769,κουναλακης εμμανουηλ πετρος,1997,συνασπισμος της αριστερας των κινηματων και τη...,NaN,μιλήσω επανάληψη άρθρο επιτροπή και τηλεγραφικ...,185,"(100, 400]"
9849,δρεττακης γεωργιου εμμανουηλ,1990,συνασπισμος της αριστερας των κινηματων και τη...,ε αντιπροεδρος της βουλης,απάντησή αντιλήφθηκα ό μηχανικούς τεχνικό προσ...,502,"(400, 1000]"
118639,κωνσταντοπουλος ανδρεα νικολαος,1997,συνασπισμος της αριστερας των κινηματων και τη...,προεδρος του συνασπισμου της αριστερας και της...,υποβάλλω παράκληση συζητήσω τροπολογίε νέος δη...,38,"(25, 100]"
...,...,...,...,...,...,...,...
500591,κακλαμανης μιχαηλ νικητας,2019,νεα δημοκρατια,προεδρευων,συζητώ ερώτηση υπάρξω ανοχή συντυχάκη,17,"(10, 25]"
500592,κακλαμανης μιχαηλ νικητας,2019,νεα δημοκρατια,προεδρευων,φύγω καταθέσω πρακτικά μπορέσω πάρω,16,"(10, 25]"
500593,συντυχακης δανιηλ εμμανουηλ,2019,κομμουνιστικο κομμα ελλαδας,NaN,καταστροφή προφανώς αποτέλεσμα καιρικός φαινομ...,681,"(400, 1000]"
500594,κακλαμανης μιχαηλ νικητας,2019,νεα δημοκρατια,προεδρευων,αρκώ κεντρικό κράτος δίνω χρήματα προβλέπω σύν...,101,"(100, 400]"


In [11]:
upsampled.political_party.value_counts()

πανελληνιο σοσιαλιστικο κινημα                                                     179384
νεα δημοκρατια                                                                     175572
συνασπισμος ριζοσπαστικης αριστερας                                                 52341
κομμουνιστικο κομμα ελλαδας                                                         30000
πολιτικη ανοιξη                                                                     30000
λαικος ορθοδοξος συναγερμος                                                         30000
δημοκρατικο κοινωνικο κινημα                                                        30000
ενωση κεντρωων                                                                      30000
το ποταμι                                                                           30000
δημοκρατικη συμπαραταξη (πανελληνιο σοσιαλιστικο κινημα - δημοκρατικη αριστερα)     30000
συνασπισμος της αριστερας των κινηματων και της οικολογιας                          30000
δημοκρατικ

code below keeps only 30000 speeches for the 3 most "popular" parties

In [8]:
nea_d = upsampled[upsampled.political_party == "νεα δημοκρατια"].sample(frac = 30000/175572)
pasok = upsampled[upsampled.political_party == "πανελληνιο σοσιαλιστικο κινημα"].sample(frac = 30000/179384)
suriza = upsampled[upsampled.political_party == "συνασπισμος ριζοσπαστικης αριστερας"].sample(frac = 30000/52341)
upsampled = upsampled [(upsampled.political_party != 'νεα δημοκρατια') & (upsampled.political_party != 'πανελληνιο σοσιαλιστικο κινημα') & ( upsampled.political_party != 'συνασπισμος ριζοσπαστικης αριστερας' )]
upsampled = pd.concat([upsampled, nea_d,pasok,suriza])
upsampled = upsampled.sample(frac=1)
upsampled.reset_index(inplace=True,drop=True)


In [ ]:
upsampled.political_party.value_counts()

import the necessary libraries and start modeling 

In [9]:
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


The models that are shown below are the ones that provided the best results after experimenting with different parameter values

Model no1 - Random Forest with 100 estimators(default) and max depth = 50 

In [10]:




X = upsampled['speech']
y =  upsampled['political_party']
xtrain,xtest,ytrain,ytest=train_test_split(X, y,test_size=0.30,random_state=123)
cv1 = CountVectorizer(lowercase=False)
x_train_cv1 = cv1.fit_transform(xtrain)
x_test_cv1 = cv1.transform(xtest) 



cv = TfidfTransformer()
x_train_cv2 = cv.fit_transform(x_train_cv1)
model=RandomForestClassifier(max_depth=50,min_samples_split=4,n_jobs=-1,criterion='entropy')
mod = model.fit(x_train_cv2, ytrain),


x_test_cv2 = cv.transform(x_test_cv1) 



y_pred = model.predict(x_test_cv2)

print(confusion_matrix(ytest,y_pred))
print(classification_report(ytest,y_pred))
print(accuracy_score(ytest, y_pred))





[[8329    9    0   45    3  177   12   33   21  204  125    9   95  118
    25]
 [ 108 8530    0    3    4   97    0   12    0  116  103    0    8   38
    28]
 [ 107    7 8504    5    0    1    5    0    6  148  110   17   11   86
     9]
 [ 466   21    0 7037   19  166   45   19   42  297  229   20  214  261
    30]
 [  80    4    0   10 8047   34    5    9   16  247  233   20    0  178
     9]
 [ 346   15    0    7    0 8284    4    4   21  206   65    6   69   54
    41]
 [ 606   52    2  224   81  314 4650  288   70  580  543   50  344 1057
    40]
 [ 494   15    0   75   42  232  182 6594   91  404  318   41  210  258
    58]
 [ 279   18    0   18   10   44   15   12 7784  277  230   29   16  114
    21]
 [ 857   35    7  205  117  288  839  348  132 1806 1550  107  685 1937
    33]
 [ 635   24    6  101  134  220  944  356   88 1325 2437  121  557 1887
    28]
 [  51    0    0    0    0   11    0    0    0   18   74 8872   15    9
     0]
 [1025   60   19  430   47  569 1234  55

 Model no2 SVM Classifier was a failure. It just kept running and running..

In [11]:
from sklearn import svm
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
X = upsampled['speech']
y =  upsampled['political_party']
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X,y,test_size=0.2)
Encoder = preprocessing.LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)
Test_Y

array([4, 5, 5, ..., 7, 3, 5])

In [12]:
cv1 = CountVectorizer(lowercase=False)
x_train_cv1 = cv1.fit_transform(Train_X)
x_test_cv1 = cv1.transform(Test_X) 



cv = TfidfTransformer()
x_train_cv2 = cv.fit_transform(x_train_cv1)

Model no3 - Multinomial NB 

In [35]:
from sklearn.naive_bayes import MultinomialNB


In [36]:
mNB = MultinomialNB()

In [37]:
model = mNB.fit(x_train_cv2,Train_Y)

In [28]:

x_test_cv2 = cv.transform(x_test_cv1) 



y_pred = model.predict(x_test_cv1)
print(confusion_matrix(Test_Y,y_pred))
print(classification_report(Test_Y,y_pred))
print(accuracy_score(Test_Y, y_pred))

[[4912    8  490   15   13   37   39   84  350   23   15    6   53    7
     7]
 [2347 2129  337   70    0  844    4    2  161    9   26   13   25    3
     0]
 [1282   12 3545  167   12  103   61   88   33   96  119   11  286  129
    42]
 [2815   23  709 1397   20  538   42   33  203   34   32   16   56   28
    17]
 [1840   16   97   36 3083   97   53   18   17  221  207  156   17  237
     4]
 [2327   23  424   65    0 2708   30    5  336   12    9    2   27   11
    10]
 [2532    5  445   85  140  248 1674  101   96   78  115   57   53  335
    17]
 [2894   20  591   26    8  188   47 1320  440  105   60   13   30   45
     7]
 [2178   54  220   87    0  314   43   76 2957   57   19   10   44   11
    23]
 [3201   17  705   73  207  155  118  122  132  641  246  100   56  301
     9]
 [3011   15  759   50  152  116  127  119  101  334  737   98   72  279
     3]
 [2145    0  227    6   45   31   11    4   30    7   26 3348    0   21
     0]
 [3152   41 1023  213   23  444  173   9

Model no4 - SGD Classifier 

In [17]:
 from sklearn.linear_model import SGDClassifier

In [18]:
 clf = SGDClassifier( loss="modified_huber", penalty="l2",)

In [19]:
model =  clf.fit(x_train_cv2,Train_Y)

In [20]:
y_pred = model.predict(x_test_cv1)
print(confusion_matrix(Test_Y,y_pred))
print(classification_report(Test_Y,y_pred))
print(accuracy_score(Test_Y, y_pred))

[[3654  273  332  248   62  368   41  217  162   14   17   72   10   72
   517]
 [ 109 4799   78  194   15  434   10   57   50    6    0   21    0   18
   179]
 [  16   71 5553   55   31   62   17   60   48    0    0   12    4   18
    39]
 [ 248  233  390 3521   86  344   42  108  118    9    6   73   10   79
   696]
 [  23   88  111  102 5182   85   78   75   37   13   15   93    4  124
    69]
 [ 266  267  118  173   29 4632   35   39   89    6    0   16    4   10
   305]
 [ 372  226  285  456  328  338 2426  401  133   18   41  149   20  369
   419]
 [ 310  252  329  213   96  294   69 3552  157   20   18   85   21   51
   327]
 [  82  168  196  158   84  125   24   92 4900    2    0   62   12   42
   146]
 [ 568  375  623  517  497  437  182  577  153  328  152  455   36  694
   489]
 [ 436  291  626  507  562  327  254  521  123  117  489  516   50  710
   444]
 [  22   46    7   73   65   43   21   11   13    1    0 5487    0   82
    30]
 [ 549  498  947  771  182  679  255  47

Model no5 - Dummy  Classifier for the comparison 

In [21]:
from sklearn.dummy import DummyClassifier

In [22]:
dummy_clf = DummyClassifier()

In [23]:
model =  dummy_clf.fit(x_train_cv2,Train_Y)

/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


In [24]:
y_pred = model.predict(x_test_cv1)
print(confusion_matrix(Test_Y,y_pred))
print(classification_report(Test_Y,y_pred))
print(accuracy_score(Test_Y, y_pred))

[[373 418 411 397 413 407 421 417 378 415 406 406 397 412 388]
 [376 368 395 435 412 400 403 387 404 403 391 404 405 392 395]
 [433 437 399 357 398 388 418 376 427 385 392 397 384 391 404]
 [396 412 385 394 426 389 383 419 357 430 398 378 371 397 428]
 [417 404 364 382 411 426 421 435 431 375 394 425 407 387 420]
 [375 411 422 410 380 388 370 390 403 380 402 407 437 399 415]
 [403 404 406 376 395 418 397 409 405 372 410 425 356 422 383]
 [413 404 390 376 362 376 371 377 406 410 393 412 364 362 378]
 [399 419 421 401 389 435 398 379 436 378 409 447 422 379 381]
 [443 364 382 421 389 396 405 409 405 435 400 400 413 401 420]
 [407 391 407 391 395 418 388 406 414 380 390 392 405 380 409]
 [391 407 405 392 368 361 402 383 386 377 392 420 399 404 414]
 [414 352 411 392 412 443 412 406 405 403 449 394 400 414 373]
 [392 397 414 434 416 415 430 411 391 400 389 392 397 396 391]
 [410 396 402 403 367 378 412 397 379 429 399 421 390 397 384]]
              precision    recall  f1-score   support


save the upsampled dataset for use in Part3

In [ ]:
upsampled.to_csv('upsampled.csv')

end of Part 2 - Machine Learning